In [340]:
import yfinance as yf

import matplotlib.pyplot as plt
import math
import pandas as pd
import numpy as np
import seaborn as sns

from ta.momentum import roc
from ta.momentum import RSIIndicator,rsi
from ta.trend import wma_indicator, ema_indicator
from ta.utils import dropna

import warnings
warnings.filterwarnings('ignore')

# Trading Strategy

Combination of a momentum indicator **Coppock Curve** with a mean-reversion indictor **RSI**. 

Dual-indicator approach that leverages the strengths of both the Coppock Curve and the RSI to maximize trading opportunities. This strategy is designed to harness the Coppock Curve's ability to identify long-term trend reversals and the RSI's sensitivity to short-term price movements. This combination allows traders to align themselves with the overall market momentum while pinpointing optimal entry points for trades. Ultimately, offering a more holistic view of the market and potentially filter out false signals.


## Signals
We will use Coppock Curve and RSI to each generate a value. 1 or -1. 

### Coppock curve [1,-1]
1 is generated when Coppock Curve crosses above zero. This indicates that the momentum has shifted from negative (bearish) to positive (bullish), suggesting that it might be a good time to buy.
-1 is generated when the Coppock Curve crosses below zero. This suggests that the momentum has shifted from positive (bullish) to negative (bearish), which might indicate that it might be a good time to short.

### RSI [1,-1]
40/70 We will use the values 40 to indicate oversold and 70 to indicate overbought.
1 is generated when RSI crossed above 40 from below or if RSI >50
-1 is generated when RSI crossed below 70 from above or if RSI <50

Combining the values generated by Coppock Curve and RSI
We will be able to generate the absolute value denoting the confidence and direction of the signal

- **Strong Buy Signal**: (1,1) 2
- **Weak Buy Signal**: (1,-1) 0
- **Strong Sell Signal**: (-1,1) 0
- **Weak Sell Signal**: (-1,-1) -2

We will then convert the value generated to get the weights.

Legend :
2 = 2 weight  
0 = 1 weight
-2 = 0 weight

We will then use the weights given to the stock to generate the capital allocated for each stock.
Capital of Stock x = Weights of Stock x / Sum of weights of all stock

## Variations of Coppock Curve

1. **Coppock Curve with standard sets of 2 periods**: The standard Coppock Curve uses a 14-day period for the long Rate of Change (RoC) and an 11-day period for the short RoC. An alternative configuration might be to use a 21-day period for the long RoC and a 5-day period for the short RoC.

2. **Coppock Curve with different smoothing constants**: The standard Coppock Curve uses a 10-period Weighted Moving Average (WMA) for smoothing.
Alternative configurations might use a different period for the WMA.

3. The standard Coppock Curve uses a **Weighted Moving Average (WMA)** for smoothing.

In [341]:
# Indicator Logic

def get_cc_wma(df,roc1_n,roc2_n,wma_lookback):
    cc_wma = wma_indicator((roc(close=df['Close_Price'], window=roc1_n,fillna= False)) + (roc(close=df['Close_Price'], window=roc2_n,fillna= False)), window=wma_lookback,fillna= False)
    return cc_wma

def get_rsi(df,window):
    rsi_indicator = rsi(close=df['Close_Price'], window=window,fillna= False)
    return rsi_indicator

def CC_RSI_strategy(df,num):
    wma = 10
    for var in range(1,num+1):
        df[f'cc_{var}'] = get_cc_wma(df, 14, 11, wma)
        wma += 100
    df['rsi_14'] = get_rsi(df, 14)
    df = df.dropna()

    for var in range(1,num+1):
        cc_crossed_above_zero = False
        rsi_crossed_40_below = False
        rsi_crossed_70_above = False
        val = [0]
        for day in range(1,len(df)):
            # Generate strat
            cc_current = df.iloc[day][f'cc_{var}']
            cc_prev = df.iloc[day-1][f'cc_{var}']
            rsi_current = df.iloc[day]['rsi_14']
            rsi_prev = df.iloc[day-1]['rsi_14']
            if cc_crossed_above_zero:
                # check rsi
                if rsi_crossed_40_below or rsi_current > 50:
                    val.append(2)
                elif rsi_crossed_70_above or rsi_current < 50:
                    val.append(0)
                else:
                    val.append(-2)

                if cc_current <= 0:
                    cc_crossed_above_zero = False
                if rsi_current < 40:
                    rsi_crossed_40_below = False
                if rsi_current > 70:
                    rsi_crossed_70_above = False
                # continue
            else:
                if rsi_crossed_40_below or rsi_current > 50:
                    val.append(0)
                elif rsi_crossed_70_above or rsi_current < 50:
                    val.append(-2)
                else:
                    val.append(0)  
            # Check if CC has crossed above zero
            if cc_current > 0 and cc_prev <= 0:
                cc_crossed_above_zero = True
            else:
                cc_crossed_above_zero = False

            # Check if RSI has crossed 40 from below
            if rsi_current > 40 and rsi_prev <= 40:
                rsi_crossed_40_below = True
            else:
                rsi_crossed_40_below = False

            # Check if RSI has crossed 70 from above
            if rsi_current < 70 and rsi_prev >= 70:
                rsi_crossed_70_above= True
            else:
                rsi_crossed_70_above = False

        df[f'cc_{var}_val'] = val
    return df
    

In [342]:
class Portfolio():
    def __init__(self, stocks, capital):
        self.stocks = stocks
        self.dataframe = pd.DataFrame() #load this dataframe sequentially
        self.initial_capital = capital # this is the initial capital you want to trade with
        self.capital = capital # this capital will change depending on trades

In [343]:
class Stock():

  def __init__(self, symbol, start, end, interval, transcation_cost, strategy, verbose = True):

    self.symbol = symbol
    self.start = start
    self.end = end
    self.interval = interval
    self.transaction_cost = transcation_cost # the transaction cost for trading
    self.quantity = 0 # quantities to buy/sell
    self.position = 0 # the trades in progress, long or short
    self.trades = 0 # Number of trades
    self.verbose = verbose # if you want to see detailed output (logs)
    self.strategy = strategy # define the strategy

    self.prepare_data() # prepares the data

  def prepare_data(self):
    # since we are building a common class for all types of strategy, we will not calcualte the moving averages now.
    # we will calculate the returns though.
    # Since most strategies utilise close prices we are only factoring close price. However, you can alter acoordingly.

    stock_data = yf.Ticker(self.symbol)
    hist_stock = stock_data.history(start = self.start, end = self.end, interval = self.interval)
    bt_data = pd.DataFrame()
    bt_data["Close_Price"] = hist_stock["Close"]
    bt_data["Open_Price"] = hist_stock["Open"]
    bt_data["Return"] = np.log(bt_data["Close_Price"] / bt_data["Close_Price"].shift(1))
    bt_data = bt_data.dropna()
    self.data = bt_data
    
  def close_graph(self):
    plt.figure(figsize=(15, 5))
    plt.plot(self.data["Close_Price"] ,color='black', label='Price', linestyle='dashed')
    plt.xlabel("Days")
    plt.ylabel("Price")
    plt.title("Close Prices of {}".format(self.symbol))
    plt.legend()
    plt.grid()
    plt.show()

  def return_date_price(self, bar):

    # A bar is a unit of data at a given time, depends on the interval you choose, it provides you OHLCV and time info
    # Since we have modeled close prices, we will get the price and date

    date = str(self.data.index[bar])[:10] #First 10 contains the date elements, rest is time
    price = self.data.Close_Price.iloc[bar]
    return date, price

  def load_strategy(self,num):
    self.data = self.strategy(self.data,num)


In [344]:
# First load the stock
META = Stock(symbol = 'META', start = '2010-01-10', end = '2016-12-31', interval = '1d', transcation_cost = 0, strategy=CC_RSI_strategy, verbose=True)
AAPL = Stock(symbol = 'AAPL', start = '2010-01-10', end = '2016-12-31', interval = '1d', transcation_cost = 0, strategy=CC_RSI_strategy, verbose = True)
AMZN = Stock(symbol = 'AMZN', start = '2010-01-10', end = '2016-12-31', interval = '1d', transcation_cost = 0, strategy=CC_RSI_strategy, verbose = True)
NFLX = Stock(symbol = 'NFLX', start = '2010-01-10', end = '2016-12-31', interval = '1d', transcation_cost = 0, strategy=CC_RSI_strategy, verbose = True)
GOOGL = Stock(symbol = 'GOOGL', start = '2010-01-10', end = '2016-12-31', interval = '1d', transcation_cost = 0, strategy=CC_RSI_strategy, verbose = True)

# Load the Portfolio with stocks
CC_RSI_portfolio = Portfolio(
    {'META' : META,
    'AAPL' : AAPL,
    'AMZN' : AAPL,
    'NFLX' : NFLX,
    'GOOGL' : GOOGL
    }, 500000)

#Example of calls
# CC_RSI_portfolio.stocks['AAPL'].data
# CC_RSI_portfolio.stocks['AAPL'].load_strategy()
# CC_RSI_portfolio.stocks['AAPL'].close_graph()
# CC_RSI_portfolio.stocks['AAPL'].return_date_price(1) # return the data of day 1
# CC_RSI_portfolio.stocks['AAPL'].realised_balance(1)
# CC_RSI_portfolio.stocks['AAPL'].unrealised_balance(1)

In [345]:
CC_RSI_portfolio.stocks['META'].data

,Close_Price,Open_Price,Return
Date,,,
2012-05-21 00:00:00-04:00,34.029999,36.529999,-0.116378
2012-05-22 00:00:00-04:00,31.000000,32.610001,-0.093255
2012-05-23 00:00:00-04:00,32.000000,31.370001,0.031749
2012-05-24 00:00:00-04:00,33.029999,32.950001,0.031680
2012-05-25 00:00:00-04:00,31.910000,32.900002,-0.034497
...,...,...,...
2016-12-23 00:00:00-05:00,117.269997,117.000000,-0.001108
2016-12-27 00:00:00-05:00,118.010002,116.959999,0.006290
2016-12-28 00:00:00-05:00,116.919998,118.190002,-0.009279


In [346]:
CC_RSI_portfolio.stocks['META'].load_strategy(2)
CC_RSI_portfolio.stocks['AAPL'].load_strategy(2)
CC_RSI_portfolio.stocks['AMZN'].load_strategy(2)
CC_RSI_portfolio.stocks['NFLX'].load_strategy(2)
CC_RSI_portfolio.stocks['GOOGL'].load_strategy(2)

In [347]:
# CC_RSI_portfolio.stocks['META'].data.head()
# CC_RSI_portfolio.stocks['AAPL'].data.head()
# CC_RSI_portfolio.stocks['AMZN'].data.head()
# CC_RSI_portfolio.stocks['NFLX'].data.head()
# CC_RSI_portfolio.stocks['GOOGL'].data.head()

In [348]:
CC_RSI_portfolio.stocks['META'].data.cc_1_val.value_counts()

cc_1_val
 0    699
-2    317
 2     23
Name: count, dtype: int64

In [349]:
CC_RSI_portfolio.stocks['META'].data.cc_2_val.value_counts()

cc_2_val
 0    711
-2    323
 2      5
Name: count, dtype: int64

In [350]:
CC_RSI_portfolio.stocks['AAPL'].data.cc_1_val.value_counts()

cc_1_val
 0    946
-2    535
 2     29
Name: count, dtype: int64

In [351]:
CC_RSI_portfolio.stocks['AAPL'].data.cc_2_val.value_counts()

cc_2_val
 0    958
-2    544
 2      8
Name: count, dtype: int64

In [352]:
CC_RSI_portfolio.stocks['AMZN'].data.cc_1_val.value_counts()

cc_1_val
 0    946
-2    535
 2     29
Name: count, dtype: int64

In [353]:
CC_RSI_portfolio.stocks['NFLX'].data.cc_1_val.value_counts()

cc_1_val
 0    1020
-2     583
 2      30
Name: count, dtype: int64

In [354]:
CC_RSI_portfolio.stocks['GOOGL'].data.cc_1_val.value_counts()

cc_1_val
 0    1021
-2     579
 2      33
Name: count, dtype: int64

In [355]:
class Stock():

  def __init__(self, symbol, start, end, interval, transcation_cost, strategy, verbose = True):

    self.symbol = symbol
    self.start = start
    self.end = end
    self.interval = interval
    self.transaction_cost = transcation_cost # the transaction cost for trading
    self.quantity = 0 # quantities to buy/sell
    self.position = 0 # the trades in progress, long or short
    self.trades = 0 # Number of trades
    self.verbose = verbose # if you want to see detailed output (logs)
    self.strategy = strategy # define the strategy

    self.prepare_data() # prepares the data

  def prepare_data(self):
    # since we are building a common class for all types of strategy, we will not calcualte the moving averages now.
    # we will calculate the returns though.
    # Since most strategies utilise close prices we are only factoring close price. However, you can alter acoordingly.

    stock_data = yf.Ticker(self.symbol)
    hist_stock = stock_data.history(start = self.start, end = self.end, interval = self.interval)
    bt_data = pd.DataFrame()
    bt_data["Close_Price"] = hist_stock["Close"]
    bt_data["Open_Price"] = hist_stock["Open"]
    bt_data["Return"] = np.log(bt_data["Close_Price"] / bt_data["Close_Price"].shift(1))
    bt_data = bt_data.dropna()
    self.data = bt_data
    
  def close_graph(self):
    plt.figure(figsize=(15, 5))
    plt.plot(self.data["Close_Price"] ,color='black', label='Price', linestyle='dashed')
    plt.xlabel("Days")
    plt.ylabel("Price")
    plt.title("Close Prices of {}".format(self.symbol))
    plt.legend()
    plt.grid()
    plt.show()

  def return_date_price(self, bar):

    # A bar is a unit of data at a given time, depends on the interval you choose, it provides you OHLCV and time info
    # Since we have modeled close prices, we will get the price and date

    date = str(self.data.index[bar])[:10] #First 10 contains the date elements, rest is time
    price = self.data.Close_Price.iloc[bar]
    return date, price

  def load_strategy(self,num):
    self.data = self.strategy(self.data,num)
    


  # def realised_balance(self, bar):

  #   #Returns you the realised capital in your account at a given time period / bar

  #   date, price = self.return_date_price(bar)
  #   print("Date :{} | Realised Balance: {:0.1f}".format(date,self.capital))

  # def unrealised_balance(self, bar):

  #   #Returns you the unrealised capital (trades in progress) in your account at a given time period / bar

  #   date, price = self.return_date_price(bar)
  #   ub = self.quantity *price
  #   print("Date :{} | Unrealised Balance: {:0.1f}".format(date,ub))

  # def total_balance(self, bar):

  #   #Unrealised plus realised

  #   date, price = self.return_date_price(bar)
  #   tb = self.quantity *price + self.capital
  #  # tb = self.quantity *price + self.capital
  #   print("Date :{} | Total Balance: {:0.1f}".format(date,tb))

  # def buy_order(self,bar,quantity=None, dollar =None ):
  #   date, price = self.return_date_price(bar)
  #   if quantity == None:
  #     quantity = int(dollar/price)
  #   self.capital = self.capital - ((quantity * price)*(1 + self.transaction_cost)) # capital will be lost in buying
  #   self.quantity = self.quantity + quantity
  #   self.trades = self.trades + 1
  #   if self.verbose:
  #     print("Bought {} shares of {} at {:0.1f} per share worth {:0.1f} $".format(quantity,self.symbol, price, quantity * price))
  #     self.realised_balance(bar)
  #     self.unrealised_balance(bar)
  #     self.total_balance(bar)

  #   self.data['Strategy_Value'].iloc[bar:] = self.capital + self.quantity * self.data['Close_Price'].iloc[bar]

  # def sell_order(self,bar,quantity=None, dollar=None ):
  #   date, price = self.return_date_price(bar)
  #   if quantity == None:
  #     quantity = int(dollar/price)
  #   self.capital = self.capital + ((quantity * price)*(1 - self.transaction_cost)) # capital will be added after selling
  #   self.quantity = self.quantity - quantity
  #   self.trades = self.trades + 1
  #   if self.verbose:
  #     print("Sold {} shares of {} at {:0.1f} per share worth {:0.1f} $".format(quantity,self.symbol, price, quantity * price))
  #     self.realised_balance(bar)
  #     self.unrealised_balance(bar)
  #     self.total_balance(bar)
  #   self.data['Strategy_Value'].iloc[bar:] = self.capital + self.quantity * self.data['Close_Price'].iloc[bar]

  # def compute_sharpe_ratio(self):
  #     avg_returns = self.data['Strategy_Returns'].mean()
  #     std = self.data['Strategy_Returns'].std()
  #     daily_SR = avg_returns / std
  #     annual_SR = daily_SR * (252 ** 0.5)
  #     return round(annual_SR, 2)
  
  # def compute_drawdown(self):
  #     cum_returns = (1 + self.data['Strategy_Returns']).cumprod()
  #     peak = cum_returns.expanding(min_periods=1).max()
  #     drawdown = (cum_returns / peak) - 1
  #     return round(drawdown.min(), 2)

  # def compute_calmar_ratio(self):
  #     annual_return = (1 + self.data['Strategy_Returns']).prod() ** (252 / len(self.data)) - 1
  #     max_drawdown = self.compute_drawdown()
  #     if max_drawdown < 0:
  #         calmar = annual_return / abs(max_drawdown)
  #     else:
  #         calmar = np.nan
  #     return round(calmar, 2)


  # def last_trade(self, bar):
  #   date, price = self.return_date_price(bar)
  #   last_quantity = self.quantity # this variable to print and store as self.quantity will be set to 0 later
  #   self.capital = self.capital + self.quantity * price
  #   self.quantity = 0 # as no more quantity now. all will be settled
  #   self.trades = self.trades +1
  #   self.data['Strategy_Value'].iloc[bar:] = self.capital

  #   if self.verbose:
  #     print("Closed open trades for {} shares of {} at {:0.1f} per share worth {:0.1f} $".format(last_quantity,self.symbol, price, last_quantity * price))
  #     self.total_balance(bar)

  #     returns = (self.capital - self.initial_capital) /self.initial_capital *100
  #     print("The total capital at end of strategy: {:0.1f}".format(self.capital))
  #     print( "The strategy returns on investment are {:0.1f} %".format(returns))
  #     print( "Total trades by startegy are {:0.1f}".format(self.trades))
  #     print( "Sharpe Ratio: {:0.1f}".format(self.compute_sharpe_ratio()))
  #     print( "Max Drawdown: {:0.1f}".format(self.compute_drawdown()))
  #     print( "Calmar Ratio: {:0.1f}".format(self.compute_calmar_ratio()))


In [356]:
class Momentum_Strategy(Common_Class):

  def go_long(self, bar, quantity = None, dollar = None):
    if self.position == -1:
      self.buy_order(bar, quantity = -self.quantity) # to clear previous short position and therefore negative quantity
    if quantity:
      self.buy_order(bar, quantity=quantity) # to create new fresh order
    elif dollar:
      if dollar == "all":
        dollar = self.capital
      self.buy_order(bar,dollar=dollar)
      
  def go_short(self, bar, quantity = None, dollar = None):
    if self.position == 1:
      self.sell_order(bar, quantity = self.quantity) # to clear previous long position
    if quantity:
      self.sell_order(bar, quantity=quantity) # to create new fresh order
    elif dollar:
      if dollar == "all":
        dollar = self.capital
      self.sell_order(bar,dollar=dollar)  

  def run_strategy(self, STMA_window, LTMA_window):
    self.position = 0
    self.trades = 0
    self.capital = self.initial_capital
    indicator_1 = SMAIndicator (close = self.data["Close_Price"], window = STMA_window, fillna= False)
    STMA = indicator_1.sma_indicator()
    indicator_2 = SMAIndicator (close = self.data["Close_Price"], window = LTMA_window, fillna= False)
    LTMA = indicator_2.sma_indicator ()

    self.data["STMA"] = STMA
    self.data["LTMA"] = LTMA

    for bar in range(LTMA_window, len(self.data)):
      if self.position in [0, -1]: # checking no position or short position 1f self. data ["STMA*].11oc [bar]›self. data ["LTMA"].11oc [bar]: self.go_long(bar, dollar-"all") # go with all money self.position - 1 # long created print("
        if self.data['STMA'].iloc[bar] > self.data['LTMA'].iloc[bar]:
          self.go_long(bar, dollar = "all")
          self.position = 1
          print('-------')
      if self.position in [0,1]:
        if self.data['STMA'].iloc[bar] < self.data['LTMA'].iloc[bar]:
          self.go_short(bar,dollar = "all")
          self.position = -1
          print('-------')
    print('-------')

    self.data['Strategy_Returns'] = self.data['Strategy_Value'].pct_change()

    self.last_trade(bar)
    

In [357]:
META = Momentum_Strategy(symbol = 'META', start = '2010-01-10', end = '2016-12-31', interval = '1d', capital = 100000, transcation_cost = 0, verbose = True)
META.data

,Close_Price,Return,Strategy_Value
Date,,,
2012-05-21 00:00:00-04:00,34.029999,-0.116378,100000.0
2012-05-22 00:00:00-04:00,31.000000,-0.093255,100000.0
2012-05-23 00:00:00-04:00,32.000000,0.031749,100000.0
2012-05-24 00:00:00-04:00,33.029999,0.031680,100000.0
2012-05-25 00:00:00-04:00,31.910000,-0.034497,100000.0
...,...,...,...
2016-12-23 00:00:00-05:00,117.269997,-0.001108,100000.0
2016-12-27 00:00:00-05:00,118.010002,0.006290,100000.0
2016-12-28 00:00:00-05:00,116.919998,-0.009279,100000.0
